In [1]:
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import keyring
import time

## Setup Spotipy credentials and query wrapper

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id=keyring.get_password('spotify', 'cid'),
                                                      client_secret=keyring.get_password('spotify', 'secret') )
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Get sample artists data

In [3]:
artist_id = '29zSTMejPhY0m7kwNQ9SPI'

In [4]:
# View sp.artist output
sp.artist(artist_id)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/29zSTMejPhY0m7kwNQ9SPI'},
 'followers': {'href': None, 'total': 75309},
 'genres': ['pinoy indie', 'pinoy rock'],
 'href': 'https://api.spotify.com/v1/artists/29zSTMejPhY0m7kwNQ9SPI',
 'id': '29zSTMejPhY0m7kwNQ9SPI',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/7451a2054f49ba2d6683bbfcd7e370ba6db575b0',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/0f7796859b8f85f23cd52c57bc9040822c01e643',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/889acd06299763826cea14a78f4b7f9720daac64',
   'width': 160}],
 'name': 'Lola Amour',
 'popularity': 48,
 'type': 'artist',
 'uri': 'spotify:artist:29zSTMejPhY0m7kwNQ9SPI'}

In [8]:
artist_albums = sp.artist_albums(artist_id)
album_ids = []
for item in artist_albums['items']:
    if album_ids:
        album_ids.append(item['id'])
    else:
        album_ids = [item['id']]
album_ids

['5ft2hxnN8ZiOiusjSK3d1h',
 '47C3r7fnfLkZdsfaAGwPRL',
 '6y3WPaQdgMfX4Mw3hhAocu',
 '388xKLbygotQvTqA0FMuwf',
 '6f2JnRkQ490vxmTKnfxe5L',
 '3JeMA5cIY58m31aXTx1hUr',
 '2uX28zFrqZw4IbzsaEYaDm',
 '652wi7TL70B3QnNE2wwDSN',
 '5eawUz4lU3YG3j79LhubBG',
 '2Lokk4sIwvne6xEXHGClBQ',
 '5D07XenBPdFIXWCj5vrpWC']

In [9]:
sp.album_tracks('5ft2hxnN8ZiOiusjSK3d1h')

{'href': 'https://api.spotify.com/v1/albums/5ft2hxnN8ZiOiusjSK3d1h/tracks?offset=0&limit=50',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/29zSTMejPhY0m7kwNQ9SPI'},
     'href': 'https://api.spotify.com/v1/artists/29zSTMejPhY0m7kwNQ9SPI',
     'id': '29zSTMejPhY0m7kwNQ9SPI',
     'name': 'Lola Amour',
     'type': 'artist',
     'uri': 'spotify:artist:29zSTMejPhY0m7kwNQ9SPI'}],
   'available_markets': ['AD',
    'AE',
    'AL',
    'AR',
    'AT',
    'AU',
    'BA',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'BY',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HR',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'JP',
    'KR',
    'KW',
    'KZ',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'M

## Get artist's album discography

In [39]:
def get_discography(a_id):
    artist_albums = sp.artist_albums(a_id)
    artist_name = sp.artist(a_id)['name']
    
    # Add album ids to list
    album_ids = []
    for item in artist_albums['items']:
        album = {'id': item['id'], 'name':item['name']}
        if album_ids:
            album_ids.append(album)
        else:
            album_ids = [album]
    
    # Get tracks from each album
    album_data = []
    for album in album_ids:
        track_data = sp.album_tracks(album['id'])['items']
        for track in track_data:
            row = [artist_id,
                   artist_name,
                   album['id'],
                   album['name'],
                   track['id'],
                   track['name'],
                   track['duration_ms'],
                   track['track_number']]
            if album_data:
                album_data.append(row)
            else:
                album_data = [row]
                
    return album_data

In [40]:
artist_discography = pd.DataFrame(get_discography(artist_id), columns=['artist_id','artist_name','album_id','album_name',
                                                  'track_id','track_name','track_length','track_number'])

In [42]:
artist_discography.to_csv('../data/artist_discography.csv', index=False, encoding='utf-8')

## Resources
- Spotify API reference manual https://developer.spotify.com/documentation/web-api/reference/search/search/